In [1]:
import nilearn
import nibabel as nib
import os
import numpy as np
import pandas as pd

In [2]:
from nilearn import datasets, image
from nilearn.maskers import NiftiMasker
from nilearn.plotting import plot_epi, plot_roi, show
from nilearn.image import resample_to_img
from nilearn.maskers import NiftiLabelsMasker
import warnings

In [3]:
from nilearn.datasets import (load_mni152_template,)
mni_template = load_mni152_template(resolution=3)

atlas=nilearn.datasets.fetch_atlas_schaefer_2018(n_rois=1000, yeo_networks=17, resolution_mm=1, data_dir=None, base_url=None, resume=True, verbose=1)
print(f"The atlas contains {len(atlas.labels) - 1} non-overlapping regions")

The atlas contains 999 non-overlapping regions


In [4]:
import matplotlib.pyplot as plt
import ants
import warnings
from tqdm import tqdm

from nilearn import datasets

In [5]:
root = r'/mnt/k/Abide Dataset/'

In [6]:
def fetch_paths(root):
    root = os.path.join(root, 'dataset/transfer - Copy - Copy/')
    fmri_data_list = {}
    universities = os.listdir(root)
    for university in universities[:1]:
        directory = os.path.join(root, university)
    
        FolderList = os.listdir(directory)
    
        for i, patient_id in enumerate(FolderList):
            if i == 5:
                
            rest_file = os.path.join(directory, patient_id, r'session_1/rest_1/rest.nii')
            fmri_data_list[patient_id] = rest_file
                
    return fmri_data_list

In [7]:
subjects = fetch_paths(root)

In [8]:
faulty_subjects = []

In [9]:
print(len(subjects))

54


In [10]:
def cal_roi(subject_id, fmri_image):
    warnings.filterwarnings('ignore', category=UserWarning)
    registered_volumes = []
    try:
        print('trying', subject_id)
        for time_idx in tqdm(range(fmri_image.shape[-1]), desc="Registering frames"):
            # Extract the 3D volume for the current timestamp
            fmri_volume = nib.Nifti1Image(fmri_image.dataobj[..., time_idx], affine=fmri_image.affine)
            
            # Resample the 3D volume to match the MNI template resolution
            resampled_volume = resample_to_img(fmri_volume, mni_template)
            
            # Convert the resampled volume to an ANTs image for registration
            resampled_volume_data = np.array(resampled_volume.dataobj)
            resampled_affine = resampled_volume.affine
            ants_volume = ants.from_numpy(resampled_volume_data)
            ants_volume.set_spacing(tuple(np.sqrt(np.sum(resampled_affine[:3, :3] ** 2, axis=0))))
            
            # Convert the MNI template to an ANTs image (if not already done)
            if time_idx == 0:
                mni_template_data = np.array(mni_template.dataobj)
                mni_affine = mni_template.affine
                ants_mni_template = ants.from_numpy(mni_template_data)
                ants_mni_template.set_spacing(tuple(np.sqrt(np.sum(mni_affine[:3, :3] ** 2, axis=0))))
            
            # Register the current volume to the MNI template
            registration_result = ants.registration(fixed=ants_mni_template, moving=ants_volume, type_of_transform='SyN')
            
            # Append the registered volume to the list
            registered_volumes.append(registration_result['warpedmovout'].numpy())
        
        # Stack all registered volumes along the time axis to create a new 4D image
        registered_4d_data = np.stack(registered_volumes, axis=-1)
        registered_4d_image = nib.Nifti1Image(registered_4d_data, affine=mni_template.affine)
    
        masker = NiftiLabelsMasker(
        atlas.maps,
        labels=labels,
        standardize="zscore_sample",
        )
        masker.fit(registered_4d_image)
        signals = masker.transform(registered_4d_image)
        signals_df = pd.DataFrame(signals)
    
        # Save the DataFrame to a CSV file
        output_path = f"{output_dir}/{subject_id}.csv"  # Adjust the path if needed
        signals_df.to_csv(output_path, index=False)

        print("done for subject", subject_id)
        
    except:
        faulty_subjects.append(subject_id)
        print('Did not run for subject', subject_id)

In [11]:
output_dir = "Unprocessed"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

labels = np.insert(atlas.labels, 0, 'Background')

# Step 4: Convert all labels to strings
labels = [str(label) for label in labels]

In [12]:
# subjects_done = os.listdir(output_dir)
# left_subjects = {} 
# for key, val in subjects:
#     if os.path.join(key, '.csv') not in list(os.listdir(output_dir)):
#         left_subjects[key] = val

In [13]:
from joblib import Parallel, delayed

import nibabel as nib

warnings.filterwarnings('ignore', category=UserWarning)

# Assuming `subjects` is a dictionary where keys are subject IDs and values are file paths
results = Parallel(n_jobs=10)(
    delayed(cal_roi)(subject_id, nib.load(path)) for subject_id, path in subjects.items()
)


trying 29864
trying 29865
trying 29866
trying 29869
trying 29868
trying 29867


Registering frames:   0%|          | 0/160 [00:00<?, ?it/s]

trying 29870
trying 29871
trying 29872
trying 29873


Registering frames:   0%|          | 0/160 [00:00<?, ?it/s]

Did not run for subject 29864
trying 29874


KeyboardInterrupt: 